In [2]:
from __future__ import print_function, division

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

class BIGAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # Build the encoder
        self.encoder = self.build_encoder()

        # The part of the bigan that trains the discriminator and encoder
        self.discriminator.trainable = False

        # Generate image from sampled noise
        z = Input(shape=(self.latent_dim, ))
        img_ = self.generator(z)

        # Encode image
        img = Input(shape=self.img_shape)
        z_ = self.encoder(img)

        # Latent -> img is fake, and img -> latent is valid
        fake = self.discriminator([z, img_])
        valid = self.discriminator([z_, img])

        # Set up and compile the combined model
        # Trains generator to fool the discriminator
        self.bigan_generator = Model([z, img], [fake, valid])
        self.bigan_generator.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
            optimizer=optimizer)

    def build_encoder(self):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(self.latent_dim))

        model.summary()
        img = Input(shape=self.img_shape)
        z = model(img)

        return Model(img, z)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()
        z = Input(shape=(self.latent_dim,))
        gen_img = model(z)

        return Model(z, gen_img)

    def build_discriminator(self):

        z = Input(shape=(self.latent_dim,))
        img = Input(shape=self.img_shape)
        d_in = concatenate([z, Flatten()(img)])

        model = Dense(32 * 32 * 3)(d_in)
        model = Reshape((32, 32, 3))(model)
        model = Conv2D(32, kernel_size=3, strides=2, padding="same")(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.25)(model)
        model = Conv2D(64, kernel_size=3, strides=2, padding="same")(model)
        model = ZeroPadding2D(padding=((0, 1), (0, 1)))(model)
        model = BatchNormalization(momentum=0.8)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.25)(model)
        model = Conv2D(128, kernel_size=3, strides=2, padding="same")(model)
        model = BatchNormalization(momentum=0.8)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.25)(model)
        model = Conv2D(256, kernel_size=3, strides=1, padding="same")(model)
        model = BatchNormalization(momentum=0.8)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.25)(model)
        model = Flatten()(model)
        validity = Dense(1, activation="sigmoid")(model)

        return Model([z, img], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            # idx = np.random.randint(0, X_train.shape[0], batch_size)
            for index in range(nb_batches):
                global_step += 1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                z = np.random.normal(size=(batch_size, self.latent_dim))
                imgs_ = self.generator.predict(z)

                # Select a random batch of images and encode
                # idx = np.random.randint(0, X_train.shape[0], batch_size)
                # imgs = X_train[idx]
                z_ = self.encoder.predict(image_batch)

                # Train the discriminator (img -> z is valid, z -> img is fake)
                d_loss_real = self.discriminator.train_on_batch([z_, image_batch], valid)
                d_loss_fake = self.discriminator.train_on_batch([z, imgs_], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (z -> img is valid and img -> z is is invalid)
                g_loss = self.bigan_generator.train_on_batch([z, image_batch], [valid, fake])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, global_step, d_loss[0],
                                                                                   100 * d_loss[1], g_loss[0]))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_interval(epoch, global_step)

    def sample_interval(self, epoch, global_step):
        r, c = 10, 10
        z = np.random.normal(size=(r*c, self.latent_dim))
        generated_images = self.generator.predict(z)

        generated_images = np.asarray((generated_images * 127.5 + 127.5).astype(np.uint8))

        def vis_square(data, padsize=1, padval=0):
            # force the number of filters to be square
            n = int(np.ceil(np.sqrt(data.shape[0])))
            padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
            data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))

            # tile the filters into an image
            data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
            data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
            return data

        img = vis_square(generated_images)
        if not os.path.isdir('images_bigan_cifar10'):
            os.mkdir('images_bigan_cifar10')
        Image.fromarray(img).save(
            "images_bigan_cifar10/epoch_%d_step_%d.png" % (epoch,global_step))

if __name__ == '__main__':
    bigan = BIGAN()
    bigan.train(epochs=50, batch_size=64, sample_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 8192)              827392    
_________________________________________________________________
reshape_3 (Reshape)          (None, 8, 8, 128)         0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 128)       147584    
_________________________________________________________________
batch_normalization_9 (Batch (None, 16, 16, 128)       512       
_________________________________________________________________
activation_4 (Activation)    (None, 16, 16, 128)       0         
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 32, 32, 128)       0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 1.125763, acc: 30.47%] [G loss: 3.417828]
epoch:0 step:2 [D loss: 1.979551, acc: 40.62%] [G loss: 4.960042]
epoch:0 step:3 [D loss: 0.517309, acc: 79.69%] [G loss: 5.826185]
epoch:0 step:4 [D loss: 0.834442, acc: 64.84%] [G loss: 5.790206]
epoch:0 step:5 [D loss: 0.344891, acc: 84.38%] [G loss: 5.412471]
epoch:0 step:6 [D loss: 0.743025, acc: 60.16%] [G loss: 4.616166]
epoch:0 step:7 [D loss: 0.435030, acc: 78.12%] [G loss: 3.797035]
epoch:0 step:8 [D loss: 0.350672, acc: 86.72%] [G loss: 3.717617]
epoch:0 step:9 [D loss: 0.348214, acc: 84.38%] [G loss: 4.215730]
epoch:0 step:10 [D loss: 0.563299, acc: 67.97%] [G loss: 5.673571]
epoch:0 step:11 [D loss: 1.143475, acc: 41.41%] [G loss: 7.037165]
epoch:0 step:12 [D loss: 0.248244, acc: 86.72%] [G loss: 7.564476]
epoch:0 step:13 [D loss: 0.108910, acc: 97.66%] [G loss: 6.433130]
epoch:0 step:14 [D loss: 0.087989, acc: 97.66%] [G loss: 4.980158]
epoch:0 step:15 [D loss: 0.102026, acc: 98.44%] [G loss: 4.364975]
epoc

epoch:0 step:124 [D loss: 0.029537, acc: 100.00%] [G loss: 10.354588]
epoch:0 step:125 [D loss: 0.033013, acc: 99.22%] [G loss: 10.506847]
epoch:0 step:126 [D loss: 0.040469, acc: 97.66%] [G loss: 10.076322]
epoch:0 step:127 [D loss: 0.013000, acc: 100.00%] [G loss: 7.502301]
epoch:0 step:128 [D loss: 0.068632, acc: 97.66%] [G loss: 6.317908]
epoch:0 step:129 [D loss: 0.047729, acc: 99.22%] [G loss: 5.223455]
epoch:0 step:130 [D loss: 0.027421, acc: 100.00%] [G loss: 6.204920]
epoch:0 step:131 [D loss: 0.019635, acc: 100.00%] [G loss: 6.177033]
epoch:0 step:132 [D loss: 0.522573, acc: 82.81%] [G loss: 8.658902]
epoch:0 step:133 [D loss: 0.028768, acc: 100.00%] [G loss: 7.759007]
epoch:0 step:134 [D loss: 0.018985, acc: 100.00%] [G loss: 7.094817]
epoch:0 step:135 [D loss: 0.025468, acc: 100.00%] [G loss: 4.997373]
epoch:0 step:136 [D loss: 0.008116, acc: 100.00%] [G loss: 5.668641]
epoch:0 step:137 [D loss: 0.009404, acc: 100.00%] [G loss: 5.697718]
epoch:0 step:138 [D loss: 0.011625, 

epoch:0 step:246 [D loss: 0.410401, acc: 81.25%] [G loss: 5.120560]
epoch:0 step:247 [D loss: 0.107949, acc: 97.66%] [G loss: 4.422062]
epoch:0 step:248 [D loss: 0.114924, acc: 96.88%] [G loss: 4.290256]
epoch:0 step:249 [D loss: 0.257575, acc: 90.62%] [G loss: 3.456339]
epoch:0 step:250 [D loss: 0.582929, acc: 75.00%] [G loss: 4.057846]
epoch:0 step:251 [D loss: 0.369617, acc: 86.72%] [G loss: 4.524366]
epoch:0 step:252 [D loss: 0.314731, acc: 85.94%] [G loss: 3.776088]
epoch:0 step:253 [D loss: 0.555812, acc: 70.31%] [G loss: 4.215684]
epoch:0 step:254 [D loss: 0.215031, acc: 93.75%] [G loss: 4.896338]
epoch:0 step:255 [D loss: 0.643550, acc: 71.88%] [G loss: 4.475292]
epoch:0 step:256 [D loss: 0.669115, acc: 66.41%] [G loss: 4.896208]
epoch:0 step:257 [D loss: 0.803855, acc: 66.41%] [G loss: 6.539287]
epoch:0 step:258 [D loss: 0.691430, acc: 80.47%] [G loss: 5.808708]
epoch:0 step:259 [D loss: 0.824583, acc: 65.62%] [G loss: 5.470138]
epoch:0 step:260 [D loss: 0.157656, acc: 96.88%]

...

epoch:49 step:38485 [D loss: 0.208620, acc: 96.09%] [G loss: 5.298833]
epoch:49 step:38486 [D loss: 0.113674, acc: 98.44%] [G loss: 8.741997]
epoch:49 step:38487 [D loss: 0.053399, acc: 100.00%] [G loss: 4.126187]
epoch:49 step:38488 [D loss: 0.168738, acc: 97.66%] [G loss: 6.619267]
epoch:49 step:38489 [D loss: 0.170117, acc: 100.00%] [G loss: 3.991983]
epoch:49 step:38490 [D loss: 0.123856, acc: 100.00%] [G loss: 4.410958]
epoch:49 step:38491 [D loss: 0.994063, acc: 50.78%] [G loss: 7.601133]
epoch:49 step:38492 [D loss: 0.315539, acc: 85.94%] [G loss: 3.228077]
epoch:49 step:38493 [D loss: 0.122353, acc: 99.22%] [G loss: 8.190208]
epoch:49 step:38494 [D loss: 0.166754, acc: 99.22%] [G loss: 2.041938]
epoch:49 step:38495 [D loss: 0.048760, acc: 100.00%] [G loss: 3.594729]
epoch:49 step:38496 [D loss: 0.123447, acc: 99.22%] [G loss: 5.096913]
epoch:49 step:38497 [D loss: 0.164486, acc: 99.22%] [G loss: 4.346774]
epoch:49 step:38498 [D loss: 0.150741, acc: 97.66%] [G loss: 8.161093]
ep

epoch:49 step:38602 [D loss: 0.041012, acc: 100.00%] [G loss: 6.682612]
epoch:49 step:38603 [D loss: 0.073029, acc: 100.00%] [G loss: 6.975055]
epoch:49 step:38604 [D loss: 0.288087, acc: 88.28%] [G loss: 3.926732]
epoch:49 step:38605 [D loss: 0.040220, acc: 100.00%] [G loss: 6.265657]
epoch:49 step:38606 [D loss: 0.376593, acc: 76.56%] [G loss: 6.817908]
epoch:49 step:38607 [D loss: 0.785327, acc: 53.12%] [G loss: 8.859297]
epoch:49 step:38608 [D loss: 0.045576, acc: 100.00%] [G loss: 7.728898]
epoch:49 step:38609 [D loss: 0.428489, acc: 89.06%] [G loss: 3.107597]
epoch:49 step:38610 [D loss: 1.016035, acc: 50.00%] [G loss: 6.948203]
epoch:49 step:38611 [D loss: 0.232984, acc: 92.97%] [G loss: 3.482243]
epoch:49 step:38612 [D loss: 0.019877, acc: 100.00%] [G loss: 9.520734]
epoch:49 step:38613 [D loss: 0.701399, acc: 56.25%] [G loss: 5.495139]
epoch:49 step:38614 [D loss: 0.502794, acc: 63.28%] [G loss: 7.050310]
epoch:49 step:38615 [D loss: 0.064051, acc: 100.00%] [G loss: 1.855579]


epoch:49 step:38718 [D loss: 0.068820, acc: 99.22%] [G loss: 8.604400]
epoch:49 step:38719 [D loss: 0.164810, acc: 97.66%] [G loss: 8.003288]
epoch:49 step:38720 [D loss: 0.095342, acc: 99.22%] [G loss: 6.035913]
epoch:49 step:38721 [D loss: 0.213181, acc: 99.22%] [G loss: 7.354232]
epoch:49 step:38722 [D loss: 0.049886, acc: 100.00%] [G loss: 6.379041]
epoch:49 step:38723 [D loss: 0.484681, acc: 75.78%] [G loss: 6.087607]
epoch:49 step:38724 [D loss: 0.310704, acc: 95.31%] [G loss: 5.391438]
epoch:49 step:38725 [D loss: 0.685896, acc: 57.81%] [G loss: 8.374493]
epoch:49 step:38726 [D loss: 0.149275, acc: 97.66%] [G loss: 6.560239]
epoch:49 step:38727 [D loss: 0.056758, acc: 100.00%] [G loss: 9.107834]
epoch:49 step:38728 [D loss: 0.252650, acc: 92.19%] [G loss: 5.463428]
epoch:49 step:38729 [D loss: 0.102270, acc: 100.00%] [G loss: 4.914287]
epoch:49 step:38730 [D loss: 0.514609, acc: 77.34%] [G loss: 9.280899]
epoch:49 step:38731 [D loss: 0.107122, acc: 98.44%] [G loss: 5.498803]
epo

epoch:49 step:38835 [D loss: 0.464397, acc: 81.25%] [G loss: 6.284185]
epoch:49 step:38836 [D loss: 0.079180, acc: 99.22%] [G loss: 6.355426]
epoch:49 step:38837 [D loss: 0.182601, acc: 99.22%] [G loss: 5.712676]
epoch:49 step:38838 [D loss: 0.583070, acc: 67.97%] [G loss: 9.178040]
epoch:49 step:38839 [D loss: 0.509483, acc: 65.62%] [G loss: 8.551725]
epoch:49 step:38840 [D loss: 0.200219, acc: 96.09%] [G loss: 5.036223]
epoch:49 step:38841 [D loss: 0.097369, acc: 100.00%] [G loss: 5.261866]
epoch:49 step:38842 [D loss: 0.686384, acc: 57.81%] [G loss: 6.913753]
epoch:49 step:38843 [D loss: 0.049907, acc: 100.00%] [G loss: 5.639013]
epoch:49 step:38844 [D loss: 0.244501, acc: 95.31%] [G loss: 1.888864]
epoch:49 step:38845 [D loss: 0.083843, acc: 100.00%] [G loss: 8.442393]
epoch:49 step:38846 [D loss: 0.655819, acc: 58.59%] [G loss: 6.186566]
epoch:49 step:38847 [D loss: 0.004490, acc: 100.00%] [G loss: 7.129922]
epoch:49 step:38848 [D loss: 0.347741, acc: 79.69%] [G loss: 9.354444]
ep

epoch:49 step:38953 [D loss: 0.146606, acc: 97.66%] [G loss: 4.495331]
epoch:49 step:38954 [D loss: 0.021447, acc: 100.00%] [G loss: 5.923525]
epoch:49 step:38955 [D loss: 0.250912, acc: 96.09%] [G loss: 7.293523]
epoch:49 step:38956 [D loss: 1.138623, acc: 26.56%] [G loss: 7.429673]
epoch:49 step:38957 [D loss: 0.113422, acc: 98.44%] [G loss: 8.368131]
epoch:49 step:38958 [D loss: 0.108716, acc: 99.22%] [G loss: 7.030182]
epoch:49 step:38959 [D loss: 0.150770, acc: 98.44%] [G loss: 3.482654]
epoch:49 step:38960 [D loss: 0.082823, acc: 100.00%] [G loss: 8.817965]
epoch:49 step:38961 [D loss: 0.055348, acc: 100.00%] [G loss: 5.920137]
epoch:49 step:38962 [D loss: 0.316173, acc: 82.81%] [G loss: 4.563265]
epoch:49 step:38963 [D loss: 0.156637, acc: 96.09%] [G loss: 4.879123]
epoch:49 step:38964 [D loss: 0.036984, acc: 100.00%] [G loss: 5.734174]
epoch:49 step:38965 [D loss: 0.283293, acc: 88.28%] [G loss: 7.586598]
epoch:49 step:38966 [D loss: 0.129493, acc: 98.44%] [G loss: 8.404425]
ep